# App Garden and Tool Shed Management

This notebook demonstrates how to list and manage deployed applications and Tool servers in Kamiwaza.

## Overview

Kamiwaza provides two deployment platforms:
- **App Garden**: Deploy containerized applications (web apps, databases, etc.)
- **Tool Shed**: Deploy MCP (Model Context Protocol) servers that provide tools to AI assistants

Let's explore how to view what's currently deployed in your Kamiwaza instance.

## Setup

First, let's import the necessary libraries and initialize the Kamiwaza client.

**Note on Authentication**: The Tool Shed endpoints require authentication, while App Garden endpoints do not. We'll use username/password authentication to access both services. The SDK automatically handles both Authorization headers and cookies for compatibility with different Kamiwaza endpoints.

In production, you should use environment variables or secure credential storage instead of hardcoding credentials.

In [ ]:
from kamiwaza_sdk import KamiwazaClient as kz
from kamiwaza_sdk.authentication import UserPasswordAuthenticator
import os
from datetime import datetime

# Initialize the client without authentication first (needed for auth service)
base_url = os.getenv("KAMIWAZA_API_URI", "http://localhost:7777/api")
client = kz(base_url=base_url)

# Create authenticator with username/password
# Note: In production, use environment variables or secure credential storage
authenticator = UserPasswordAuthenticator(
    username="admin",
    password="kamiwaza",
    auth_service=client.auth
)

# Re-initialize client with authentication
client = kz(
    base_url=base_url,
    authenticator=authenticator
)

print(f"Connected to Kamiwaza at: {client.base_url}")
print("✅ Authenticated as: admin")

## App Garden - List Deployed Applications

The App Garden allows you to deploy containerized applications. Let's see what's currently running:

In [ ]:
# List all deployed applications
app_deployments = client.apps.list_deployments()

print(f"\n🌱 App Garden - {len(app_deployments)} application(s) deployed:\n")

if not app_deployments:
    print("No applications currently deployed.")
else:
    for app in app_deployments:
        print(f"📦 {app.name}")
        print(f"   ID: {app.id}")
        print(f"   Status: {app.status}")
        print(f"   Created: {app.created_at.strftime('%Y-%m-%d %H:%M:%S')}")
        if app.deployed_at:
            print(f"   Deployed: {app.deployed_at.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"   Instances: {app.min_copies} min, {app.max_copies or app.min_copies} max")
        
        # Show instance details
        instances = client.apps.list_instances(app.id)
        if instances:
            print(f"   Running instances:")
            for inst in instances:
                print(f"     - {inst.host_name}:{inst.listen_port} (Status: {inst.status})")
        
        print()

## Tool Shed - List Deployed Tool Servers

Tool servers provide MCP endpoints that AI assistants can use to access external capabilities:

In [ ]:
# List all deployed Tool servers
tool_deployments = client.tools.list_deployments()

print(f"\n🔧 Tool Shed - {len(tool_deployments)} Tool server(s) deployed:\n")

if not tool_deployments:
    print("No Tool servers currently deployed.")
else:
    for tool in tool_deployments:
        print(f"🛠️  {tool.name}")
        print(f"   ID: {tool.id}")
        print(f"   Status: {tool.status}")
        print(f"   URL: {tool.url}")
        print(f"   Created: {tool.created_at.strftime('%Y-%m-%d %H:%M:%S')}")
        
        # Check health status
        try:
            health = client.tools.check_health(tool.id)
            print(f"   Health: {health.status}")
            if health.protocol_version:
                print(f"   Protocol: {health.protocol_version}")
        except Exception as e:
            print(f"   Health: Unable to check - {str(e)}")
        
        print()

## Discover All Tool Servers and Their Capabilities

The discovery endpoint provides information about all Tool servers and their capabilities:

In [ ]:
# Discover all Tool servers
discovery = client.tools.discover_servers()

print(f"\n🔍 Tool Server Discovery - {discovery.total} server(s) found:\n")

for server in discovery.servers:
    print(f"🛠️  {server.name}")
    print(f"   URL: {server.url}")
    print(f"   Status: {server.status}")
    
    if server.capabilities:
        print(f"   Capabilities:")
        for cap in server.capabilities:
            desc = f" - {cap.description}" if cap.description else ""
            print(f"     • {cap.name}{desc}")
    else:
        print(f"   Capabilities: Not available")
    
    print()

## View Available Templates

Let's see what applications and Tool servers are available to deploy:

In [ ]:
# List available app templates
app_templates = client.apps.list_templates()

print(f"\n📋 Available App Templates ({len(app_templates)} total):\n")

for template in app_templates[:5]:  # Show first 5
    print(f"• {template.name} (v{template.version or '1.0.0'})")
    if template.description:
        print(f"  {template.description[:100]}..." if len(template.description) > 100 else f"  {template.description}")
    print(f"  Risk Tier: {template.risk_tier} | Verified: {template.verified}")
    print()

if len(app_templates) > 5:
    print(f"... and {len(app_templates) - 5} more templates")

In [ ]:
# List available Tool templates
tool_templates = client.tools.list_available_templates()

print(f"\n🔧 Available Tool Templates ({len(tool_templates)} total):\n")

for template in tool_templates:
    print(f"• {template.name}")
    print(f"  {template.description[:100]}..." if len(template.description) > 100 else f"  {template.description}")
    if template.required_env_vars:
        print(f"  Required env vars: {', '.join(template.required_env_vars)}")
    print()

## Summary Statistics

Let's create a summary of the deployment status:

In [ ]:
# Calculate summary statistics
total_apps = len(app_deployments)
total_tools = len(tool_deployments)
running_apps = sum(1 for app in app_deployments if app.status == "RUNNING")
running_tools = sum(1 for tool in tool_deployments if tool.status == "RUNNING")

print("\n📊 Deployment Summary:\n")
print(f"App Garden:")
print(f"  Total deployments: {total_apps}")
print(f"  Running: {running_apps}")
print(f"  Other states: {total_apps - running_apps}")
print(f"\nTool Shed:")
print(f"  Total deployments: {total_tools}")
print(f"  Running: {running_tools}")
print(f"  Other states: {total_tools - running_tools}")
print(f"\nTotal deployments across both services: {total_apps + total_tools}")

# Show deployment age
if app_deployments or tool_deployments:
    print("\n⏱️  Deployment Ages:")
    
    all_deployments = [("App", d) for d in app_deployments] + [("Tool", d) for d in tool_deployments]
    all_deployments.sort(key=lambda x: x[1].created_at)
    
    for dep_type, dep in all_deployments:
        age = datetime.now() - dep.created_at.replace(tzinfo=None)
        days = age.days
        hours = age.seconds // 3600
        minutes = (age.seconds % 3600) // 60
        
        age_str = ""
        if days > 0:
            age_str = f"{days}d {hours}h"
        elif hours > 0:
            age_str = f"{hours}h {minutes}m"
        else:
            age_str = f"{minutes}m"
            
        print(f"  [{dep_type}] {dep.name}: {age_str} old")

## Next Steps

Now that you can see what's deployed, you can:

1. **Deploy new applications**:
   ```python
   deployment = client.apps.deploy(
       template_id=template.id,
       name="my-app",
       env_vars={"KEY": "value"}
   )
   ```

2. **Deploy Tool servers**:
   ```python
   tool = client.tools.deploy_from_template(
       template_name="tool-websearch",
       name="search-tool",
       env_vars={"TAVILY_API_KEY": "your-key"}
   )
   ```

3. **Stop deployments**:
   ```python
   client.apps.stop_deployment(deployment_id)
   client.tools.stop_deployment(tool_id)
   ```

4. **Use Tool URLs with AI assistants** - The Tool URLs can be used with any MCP-compatible client.

Happy deploying! 🚀